In [49]:
# Install if required
!pip install medmnist

In [50]:
#Importing necessary packages

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
import medmnist
from medmnist import INFO, Evaluator

In [51]:
#Creating dataflags

data_flag = 'synapsemnist3d'
download = True
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

# load the data
train_dataset = DataClass(split='train', download=download, size=64)
test_dataset = DataClass(split='test', download=download, size=64)
val_dataset = DataClass(split='val', download=download, size=64)

Using downloaded and verified file: /root/.medmnist/synapsemnist3d_64.npz
Using downloaded and verified file: /root/.medmnist/synapsemnist3d_64.npz
Using downloaded and verified file: /root/.medmnist/synapsemnist3d_64.npz


In [52]:
test_dataset

Dataset SynapseMNIST3D of size 64 (synapsemnist3d_64)
    Number of datapoints: 352
    Root location: /root/.medmnist
    Split: test
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'inhibitory synapse', '1': 'excitatory synapse'}
    Number of samples: {'train': 1230, 'val': 177, 'test': 352}
    Description: The SynapseMNIST3D is a new 3D volume dataset to classify whether a synapse is excitatory or inhibitory. It uses a 3D image volume of an adult rat acquired by a multi-beam scanning electron microscope. The original data is of the size 100×100×100um^3 and the resolution 8×8×30nm^3, where a (30um)^3 sub-volume was used in the MitoEM dataset with dense 3D mitochondria instance segmentation labels. Three neuroscience experts segment a pyramidal neuron within the whole volume and proofread all the synapses on this neuron with excitatory/inhibitory labels. For each labeled synaptic location, we crop a 3D volume of 1024×1024×1024nm^3 and resize it into 28×

In [53]:
#Creating a function which converts a 3D image into 2D image in tensor format
def image_3d_to_2d(dataset):

  #Initializing empty lists to store 2D images
  images = []
  labels = []
  index = []
  print("Initial 3D images:",len(dataset))

  for i in range(len(dataset)):           #This loop runs for a few iterations as many as no. of 3D images
    x, y = dataset[i]               #Extracting the image tensor (x) and label (y) from one 3D image
    for j in range(len(x[0])):            #Getting into first batch of the selected 3D image. There will be only 1 batch and hence x[0] and len(x[0]) gives number of slices
      img = x[0][j]                       #Array for every slice (j) is captured
      arr = (img).astype(np.float32)      #Converting the normalized array into float
      tensor = torch.tensor(arr)          #Converting array into tensor
      images.append(tensor)               #Appending the image tensor into a list
      labels.append(y[0])                 #Appending the label for the image into a list
      index.append(i)                     #Appending the 3D image index into a list
  print("Sliced 2D images:",len(images))
  return images, labels, index

In [54]:
#Running the above function for train, test and validation.
#Since the size=28, every 3D image has 28 slices

train_2d = image_3d_to_2d(train_dataset)
test_2d = image_3d_to_2d(test_dataset)
val_2d = image_3d_to_2d(val_dataset)

Initial 3D images: 1230
Sliced 2D images: 78720
Initial 3D images: 352
Sliced 2D images: 22528
Initial 3D images: 177
Sliced 2D images: 11328


In [55]:
#Once sliced up, images are saved as .pth file

images, labels, index = train_2d
torch.save({'images': images, 'labels': labels, 'index':index}, 'synapse_train_2d.pth')
images, labels, index = test_2d
torch.save({'images': images, 'labels': labels, 'index':index}, 'synapse_test_2d.pth')
images, labels, index = val_2d
torch.save({'images': images, 'labels': labels, 'index':index}, 'synapse_val_2d.pth')

In [56]:
# Load the saved dataset
data_train = torch.load('synapse_train_2d.pth')
data_test = torch.load('synapse_test_2d.pth')
data_val = torch.load('synapse_val_2d.pth')

# Convert the loaded images and labels to tensors
tr_images = torch.stack(data_train['images'])  # Stack list of tensors into a single tensor
tr_images = tr_images.unsqueeze(1)              #Forcefully adding n_channel=1
tr_images_rgb = tr_images.repeat(1, 3, 1, 1)
tr_labels = torch.tensor(data_train['labels'])  # Convert list of labels into a tensor
# if tr_labels.ndimension() == 1:  # If it's (num_samples,)
    # num_classes = 3  # Replace with actual number of classes
    # tr_labels = torch.nn.functional.one_hot(tr_labels, num_classes=num_classes).float()
tr_index = torch.tensor(data_train['index'])  # Convert list of indices into a tensor

ts_images = torch.stack(data_test['images'])  # Stack list of tensors into a single tensor
ts_images = ts_images.unsqueeze(1)              #Forcefully adding n_channel=1
ts_images_rgb = ts_images.repeat(1, 3, 1, 1)
ts_labels = torch.tensor(data_test['labels'])  # Convert list of labels into a tensor
# if ts_labels.ndimension() == 1:  # If it's (num_samples,)
#     num_classes = 3  # Replace with actual number of classes
#     ts_labels = torch.nn.functional.one_hot(ts_labels, num_classes=num_classes).float()
ts_index = torch.tensor(data_test['index'])  # Convert list of indices into a tensor

val_images = torch.stack(data_val['images'])  # Stack list of tensors into a single tensor
val_images = val_images.unsqueeze(1)              #Forcefully adding n_channel=1
val_images_rgb = val_images.repeat(1, 3, 1, 1)
val_labels = torch.tensor(data_val['labels'])  # Convert list of labels into a tensor
val_index = torch.tensor(data_val['index'])  # Convert list of indices into a tensor

# Create a TensorDataset from the loaded data
dataset_train = data.TensorDataset(tr_images_rgb, tr_labels, tr_index)
dataset_test = data.TensorDataset(ts_images_rgb, ts_labels, ts_index)
dataset_val = data.TensorDataset(val_images_rgb, val_labels, val_index)

# Create a DataLoader for batching and shuffling
train_loader = data.DataLoader(dataset_train, batch_size=128, shuffle=True)
test_loader = data.DataLoader(dataset_test, batch_size=128, shuffle=True)
val_loader = data.DataLoader(dataset_val, batch_size=128, shuffle=True)



<ipython-input-56-4afe41148add>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_train = torch.load('synapse_train_2d.pth')
<ipython-input-56-4afe41148add>:3: FutureWar

In [57]:
# To see each batch of images
# for batch in test_loader:
#     batch_images, batch_labels, batch_index = batch
#     print(batch_images.shape)  # Should be (batch_size, 1, 28, 28)
#     print(batch_labels)

In [58]:
!nvidia-smi

Wed Sep 18 16:16:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   69C    P0              32W /  72W |    723MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [59]:
!git clone https://github.com/Omid-Nejati/MedViT.git
%cd /content/MedViT

Cloning into 'MedViT'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 176 (delta 87), reused 134 (delta 61), pack-reused 1 (from 1)
Receiving objects: 100% (176/176), 820.48 KiB | 1.75 MiB/s, done.
Resolving deltas: 100% (87/87), done.
/content/MedViT


In [60]:
pip install -r requirements.txt

In [61]:
from MedViT import MedViT_small as tiny, MedViT_base, MedViT_large

model = tiny()

initialize_weights...


In [62]:
# Assuming you're using a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the GPU
model = model.to(device)

In [63]:
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=n_classes, bias=True)

In [64]:
n_classes

2

In [65]:
model = model.to(device)

In [66]:
# evaluation

# task = 'multi-label, binary-class'
model.eval()
y_true = torch.tensor([]).cuda()
y_score = torch.tensor([]).cuda()

data_loader = test_loader

with torch.no_grad():
    for inputs, targets, index in data_loader:
        inputs, targets, index = inputs.cuda(), targets.cuda(), index.cuda()
        outputs = model(inputs)

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
            outputs = outputs.softmax(dim=-1)
            # print(1)
        else:
            targets = targets.squeeze().long()
            outputs = outputs.softmax(dim=-1)
            targets = targets.float().resize_(len(targets), 1)
            # print(0)

        y_true = torch.cat((y_true, targets), 0)
        y_score = torch.cat((y_score, outputs), 0)

    y_true = y_true.cpu().numpy()
    y_score = y_score.detach().cpu().numpy()

In [67]:
actuals = y_true.squeeze().tolist()
predictions = []
y_score = y_score.tolist()
for i in range(len(y_score)):
  predictions.append(y_score[i].index(max(y_score[i])))

In [68]:
correct_predictions = sum(a==p for a, p in zip(actuals, predictions))
accuracy = (correct_predictions / len(actuals))

In [69]:
accuracy

0.7301136363636364

In [70]:
precision = precision_score(actuals, predictions, average='macro')
recall = recall_score(actuals, predictions, average='macro')
f1 = f1_score(actuals, predictions, average='macro')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [71]:
print("Accuracy:", accuracy*100)
print("Precision:", precision*100)
print("Recall:", recall*100)
print("F1 Score:", f1*100)

Accuracy: 73.01136363636364
Precision: 36.50568181818182
Recall: 50.0
F1 Score: 42.200328407224966
